### Python script 

In [207]:
# Improted required libraries
import tweepy
import re 
import re
import emoji
# bearer token used for authenticate 
MY_BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAANdNkQEAAAAAxDkgxIjfEdTtDvruXFETgME75Us%3DL197GNXhxAUEoLWnByUSbXbApneRpUgCfARZcEtWYEcYXMFFRa"
# create your client 
client = tweepy.Client(bearer_token=MY_BEARER_TOKEN)

In [208]:
# query to search for IKEA tweets 
query =  "IKEA lang:en -is:retweet"
# your start and end time for fetching tweets
start_time = "2022-12-08T00:00:00Z"
end_time = "2022-12-09T00:00:00Z"
# get tweets from the API, here picking 100 records
tweets = client.search_recent_tweets(query=query,
                                     start_time=start_time,
                                     end_time=end_time,
                                     tweet_fields = ["created_at", "text", "source"],
                                     user_fields = ["name", "username", "location", "verified", "description"],
                                     max_results = 100,
                                     expansions='author_id'
                                     )

In [209]:
# tweet specific info
print(len(tweets.data))
# user specific info
print(len(tweets.includes["users"]))

100
97


In [210]:
# first tweet
first_tweet = tweets.data[0]
dict(first_tweet)

{'edit_history_tweet_ids': ['1601003379249270784'],
 'author_id': 1416674040090185730,
 'id': 1601003379249270784,
 'text': 'Someone rage quit at the ikea https://t.co/wV4WbAhswp',
 'source': 'Twitter for Android',
 'created_at': datetime.datetime(2022, 12, 8, 23, 58, 45, tzinfo=datetime.timezone.utc)}

In [211]:
# user information for the first tweet
first_tweet_user = tweets.includes["users"][0]
dict(first_tweet_user)

{'username': 'TrueBakkerboy',
 'id': 1416674040090185730,
 'verified': False,
 'name': 'TrueBakkerboy',
 'description': '🇨🇦\noccasional video maker\nhttps://t.co/snusR8i1V9'}

In [212]:
# Creating data frame 
# create a list of records
tweet_info_ls = []

# iterate over each tweet and corresponding user details
for tweet, user in zip(tweets.data, tweets.includes['users']):
    tweet_info = {
        'created_at': tweet.created_at,
        'text': tweet.text,
        'source': tweet.source,
        'id':tweet.id,
        'name': user.name,
        'username': user.username,
        'location': user.location,
        'verified': user.verified,
        'description': user.description
    }
    tweet_info_ls.append(tweet_info)

# create dataframe from the extracted records
Ikea_df = pd.DataFrame(tweet_info_ls)
# display the dataframe
Ikea_df.head()

,created_at,description,id,location,name,source,text,username,verified
0,2022-12-08 23:58:45+00:00,🇨🇦\noccasional video maker\nhttps://t.co/snusR...,1601003379249270784,None,TrueBakkerboy,Twitter for Android,Someone rage quit at the ikea https://t.co/wV4...,TrueBakkerboy,False
1,2022-12-08 23:56:44+00:00,waggin dragon 🌲 ONLY sfw!!! my little account ...,1601002873696837632,cave of critters,dragon is waggin,Twitter for Android,@mooshtelid the magical toilet paper roll insi...,dragonwaggin,False
2,2022-12-08 23:56:43+00:00,,1601002865866444800,None,Carla,Twitter for Android,@scrumtuous Visualizing it. All websites can h...,DolceKar,False
3,2022-12-08 23:56:36+00:00,MHB❤️💍🌹,1601002838314078208,Vatican City,Anak Yat 🪐,Twitter for iPhone,Karipap ikea is the best. Thank You 🎈 baby🤭 ht...,hillmyt,False
4,2022-12-08 23:55:48+00:00,I'm here to go to Chilly Go and follow cool pe...,1601002637347782656,None,Retvrn_Triji,Twitter Web App,"@DolioJ @GathererSkull Dude, what if they were...",ChillyBound,False


###  Inspect the dataframe

In [213]:
Ikea_df.shape

(97, 9)

In [214]:
#Checking for null values
Ikea_df.isnull().sum()

created_at      0
description     0
id              0
location       22
name            0
source          0
text            0
username        0
verified        0
dtype: int64

In [215]:
# column-wise null percentages here
round(100*(Ikea_df.isnull().sum()/len(Ikea_df.index)),2)


created_at      0.00
description     0.00
id              0.00
location       22.68
name            0.00
source          0.00
text            0.00
username        0.00
verified        0.00
dtype: float64

Location column is having 22% null value in a sample of 100. This column can be dropped based on the suggsetion from data science team.

In [216]:
Ikea_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 9 columns):
created_at     97 non-null datetime64[ns, UTC]
description    97 non-null object
id             97 non-null int64
location       75 non-null object
name           97 non-null object
source         97 non-null object
text           97 non-null object
username       97 non-null object
verified       97 non-null bool
dtypes: bool(1), datetime64[ns, UTC](1), int64(1), object(6)
memory usage: 6.2+ KB



###  Deriving new columns which helps for analysis

In [217]:
# Created hashtag column which will show the hashtag from the tweet.

Ikea_df['hashtag'] = Ikea_df['text'].apply(lambda x: re.findall(r'#(\w+)', x))

In [218]:
Ikea_df.head()

,created_at,description,id,location,name,source,text,username,verified,hashtag
0,2022-12-08 23:58:45+00:00,🇨🇦\noccasional video maker\nhttps://t.co/snusR...,1601003379249270784,None,TrueBakkerboy,Twitter for Android,Someone rage quit at the ikea https://t.co/wV4...,TrueBakkerboy,False,[]
1,2022-12-08 23:56:44+00:00,waggin dragon 🌲 ONLY sfw!!! my little account ...,1601002873696837632,cave of critters,dragon is waggin,Twitter for Android,@mooshtelid the magical toilet paper roll insi...,dragonwaggin,False,[]
2,2022-12-08 23:56:43+00:00,,1601002865866444800,None,Carla,Twitter for Android,@scrumtuous Visualizing it. All websites can h...,DolceKar,False,[]
3,2022-12-08 23:56:36+00:00,MHB❤️💍🌹,1601002838314078208,Vatican City,Anak Yat 🪐,Twitter for iPhone,Karipap ikea is the best. Thank You 🎈 baby🤭 ht...,hillmyt,False,[]
4,2022-12-08 23:55:48+00:00,I'm here to go to Chilly Go and follow cool pe...,1601002637347782656,None,Retvrn_Triji,Twitter Web App,"@DolioJ @GathererSkull Dude, what if they were...",ChillyBound,False,[]


In [219]:
# Created emojis columns which shows the emoticons from tweet
pattern = re.compile(r"|".join(map(re.escape, emoji.EMOJI_DATA)))

Ikea_df["emojis"] = Ikea_df["text"].apply(lambda x: "".join(pattern.findall(x)))
Ikea_df["text"] = Ikea_df["text"].apply(lambda x: pattern.sub("", x))
Ikea_df.head()

,created_at,description,id,location,name,source,text,username,verified,hashtag,emojis
0,2022-12-08 23:58:45+00:00,🇨🇦\noccasional video maker\nhttps://t.co/snusR...,1601003379249270784,None,TrueBakkerboy,Twitter for Android,Someone rage quit at the ikea https://t.co/wV4...,TrueBakkerboy,False,[],
1,2022-12-08 23:56:44+00:00,waggin dragon 🌲 ONLY sfw!!! my little account ...,1601002873696837632,cave of critters,dragon is waggin,Twitter for Android,@mooshtelid the magical toilet paper roll insi...,dragonwaggin,False,[],
2,2022-12-08 23:56:43+00:00,,1601002865866444800,None,Carla,Twitter for Android,@scrumtuous Visualizing it. All websites can h...,DolceKar,False,[],
3,2022-12-08 23:56:36+00:00,MHB❤️💍🌹,1601002838314078208,Vatican City,Anak Yat 🪐,Twitter for iPhone,Karipap ikea is the best. Thank You baby http...,hillmyt,False,[],🎈🤭
4,2022-12-08 23:55:48+00:00,I'm here to go to Chilly Go and follow cool pe...,1601002637347782656,None,Retvrn_Triji,Twitter Web App,"@DolioJ @GathererSkull Dude, what if they were...",ChillyBound,False,[],


### Tokenize the tweet which helps for Natural Language Processing

In [220]:
#Tokenize the tweet text and created new column 'Tokenize'
from nltk.tokenize import TweetTokenizer

tknzr = TweetTokenizer()
Ikea_df['Tokenize'] = Ikea_df['text'].apply(lambda s: tknzr.tokenize(s))



In [221]:

import string

# remove punctuations
Ikea_df['Tokenize']  = Ikea_df['Tokenize'] .apply(lambda l: [s.translate(str.maketrans('','',string.punctuation)) for s in l])

# remove noise
Ikea_df['Tokenize'] = Ikea_df['Tokenize'] .apply(lambda l: [s for s in l if s.isalpha()])
Ikea_df.head()

,created_at,description,id,location,name,source,text,username,verified,hashtag,emojis,Tokenize
0,2022-12-08 23:58:45+00:00,🇨🇦\noccasional video maker\nhttps://t.co/snusR...,1601003379249270784,None,TrueBakkerboy,Twitter for Android,Someone rage quit at the ikea https://t.co/wV4...,TrueBakkerboy,False,[],,"[Someone, rage, quit, at, the, ikea]"
1,2022-12-08 23:56:44+00:00,waggin dragon 🌲 ONLY sfw!!! my little account ...,1601002873696837632,cave of critters,dragon is waggin,Twitter for Android,@mooshtelid the magical toilet paper roll insi...,dragonwaggin,False,[],,"[mooshtelid, the, magical, toilet, paper, roll..."
2,2022-12-08 23:56:43+00:00,,1601002865866444800,None,Carla,Twitter for Android,@scrumtuous Visualizing it. All websites can h...,DolceKar,False,[],,"[scrumtuous, Visualizing, it, All, websites, c..."
3,2022-12-08 23:56:36+00:00,MHB❤️💍🌹,1601002838314078208,Vatican City,Anak Yat 🪐,Twitter for iPhone,Karipap ikea is the best. Thank You baby http...,hillmyt,False,[],🎈🤭,"[Karipap, ikea, is, the, best, Thank, You, baby]"
4,2022-12-08 23:55:48+00:00,I'm here to go to Chilly Go and follow cool pe...,1601002637347782656,None,Retvrn_Triji,Twitter Web App,"@DolioJ @GathererSkull Dude, what if they were...",ChillyBound,False,[],,"[DolioJ, GathererSkull, Dude, what, if, they, ..."
